<a href="https://colab.research.google.com/github/alyssa-tsh/CS3244_ML_Project/blob/main/cs3244_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from functions import data_pipeline
from functions_ver2 import data_pipeline as data_pipeline_v2

/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'functions_ver2'

In [2]:
train, test = data_pipeline(model_name='XGB')

Loading data
Splitting credits data
Total unique accounts: 45985. Starting to find cutoff point
Cutoff month where CDF reaches 80%: -10

=== Split based on CDF 80% cutoff ===
Cutoff month: -10 (10 months ago)
Old accounts (≤ month -10): 37,210 (80.9%)
New accounts (> month -10): 8,775 (19.1%)
Ratio (old/new): 4.2405
Splitting raw credit records
Cleaning old accounts credit records - [Length: 996586]


/workspaces/CS3244_ML_Project/functions.py:169: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = df.groupby(['id', 'origination_month']).apply(lambda x: pd.Series({


Cleaning new accounts credit records - [Length: 51989]


/workspaces/CS3244_ML_Project/functions.py:169: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = df.groupby(['id', 'origination_month']).apply(lambda x: pd.Series({
/workspaces/CS3244_ML_Project/functions.py:86: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dropped=df_sorted.groupby('id', group_keys=False).apply(keep_row)


Cleaning credit data completed
Splitting applications data
Cleaning old application records
Cleaning new application records
Merging data
Engineering target variable to label data
Completed old accounts labelling
Completed new accounts labelling
Old accounts: (37210, 3)
New accounts: (8775, 3)
Old threshold: 0.19732232232232233
New threshold: 0.5786182336182336
Merging cleaned application and credit records
Train shape: (29264, 28)
Test shape: (7193, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29264 entries, 0 to 29263
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           29264 non-null  int64  
 1   cnt_children                 29264 non-null  int64  
 2   amt_income_total             29264 non-null  float64
 3   days_birth                   29264 non-null  int64  
 4   days_employed                29264 non-null  int64  
 5   flag_mobil           

## Model Pipeline

### Functions

In [ ]:
def get_n_components(X_train, method = "avg"):
        pca = PCA()
        pca.fit(X_train)
        explained_var_ratio = pca.explained_variance_ratio_
        if method == "avg":
            avg_var = 1 / len(explained_var_ratio)
            optimal_components = np.sum(explained_var_ratio > avg_var)

        elif method == "elbow":
            diffs = np.diff(explained_var_ratio)
            elbow_idx = np.argmax(diffs * -1) + 1 
            optimal_components = elbow_idx

        elif method == "cumulative":
            cum_var = np.cumsum(explained_var_ratio)
            optimal_components = np.argmax(cum_var >= 0.95) + 1
        else:
            # comparison with a base model to see if PCA methods are actually improving the model
            optimal_components = None
        return optimal_components
    
methods = ["avg", "elbow", "cumulative", "default"]

### Transformations & Scaling
- from QQ plot analysis - noticed that certain numerical features require transformation and diff features need different scalers
> Highly skewed & outlier-heavy	risk_score, months_employed	→ YeoJohnsonTransformer() or np.log1p() → RobustScaler

> Already normal/log-transformed	amt_income_total_log, age	→ StandardScaler

> Discrete / ordinal numeric	cnt_children, cnt_fam_members	→ Keep as is or encode as ordinal integers

### Encoding
* categorical features - label encoder might assign encoded categories some inherent ordering affecting model which is fine for tree based models & XGBoost but not for SVC and KNN, so need to use diff encoding methods that suit the diff models

| Feature type                  | XGBoost                     | SVC          | KNN                       |
| ----------------------------- | --------------------------- | ------------ | ------------------------- |
| Binary                        | 0/1 mapping                 | 0/1 mapping  | 0/1 mapping               |
| Low-cardinality (<5)          | One-hot or label encoding   | One-hot only | One-hot                   |
| Medium/high-cardinality (~17) | Frequency or label encoding | One-hot only | One-hot / binary encoding |
| Numeric                       | Raw                         | Standardized | Standardized              |

## Dropping of correlated features
| Feature type     | XGBoost / Tree                      | SVC / KNN / Linear                |
| ---------------- | ----------------------------------- | --------------------------------- |
| Discrete numeric | keep numeric                        | Better as categorical / one-hot   |
| Binned/ordinal   | Optional (tree can handle either)   | Use one-hot encoding              |

## Feature Selection



In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import (
    PowerTransformer, RobustScaler, StandardScaler,
    OneHotEncoder, OrdinalEncoder, FunctionTransformer
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE, SelectFromModel

# ------------------------------------------------------------
# Column Dictionary
# ------------------------------------------------------------

numeric_cols = ["age", "cnt_children", "amt_income_total_log", "risk_score", "months_employed"]


# ------------------------------------------------------------
# 1. Build Transformer
# ------------------------------------------------------------
def build_transformer():
    
    transformers=[
            ("num", StandardScaler(), numeric_cols)
        ]

    preprocessor = ColumnTransformer(transformers, remainder='passthrough')
    
    return preprocessor
# ------------------------------------------------------------
# 2. Drop Correlated Features
# ------------------------------------------------------------
# def drop_correlated_features(model_name, col_dic=column_dic):
drop_cols = ["days_birth", "amt_income_total", "years_employed", "flag_mobil", "code_gender", "flag_own_realty", "flag_own_car", "cnt_fam_members"]
    # if model_name in ["SVC", "KNN"]:
    #     drop_cols.extend(["cnt_children", "cnt_fam_members"])
    #     drop_cols.extend
    # return drop_cols

# ------------------------------------------------------------
# 3. Feature Selection Model
# ------------------------------------------------------------
def build_feature_selector(model_name):
    if model_name=="SVC":
        return RFE(SVC(kernel='linear'), n_features_to_select=None, step=0.2, importance_getter='feature_importances_')
    elif model_name=="XGB":
        return SelectFromModel(XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42), threshold='median')
    elif model_name=="KNN":
        return SelectKBest(score_func=mutual_info_classif, k=10)
# ------------------------------------------------------------
# 4. Build Model
# ------------------------------------------------------------
def build_model(model_name):
    if model_name == "SVC":
        return "SVM (Linear)", SVC(kernel='linear', random_state=42)
    elif model_name == "XGB":
        return "XGB Classifier", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
    elif model_name == "KNN":
        return "KNN", KNeighborsClassifier()
    else:
        raise ValueError("Unsupported model name")
    


# ------------------------------------------------------------
# 5. Model Training Pipeline
# ------------------------------------------------------------
def model_pipeline(model_name, train_df, test_df, target_col="label", random_state=42):
    # Drop correlated columns
    drop_cols = ["days_birth", "amt_income_total", "years_employed", "flag_mobil", "code_gender", "flag_own_realty", "flag_own_car", "cnt_fam_members"]
    train_df = train_df.drop(columns=drop_cols, errors='ignore')
    test_df = test_df.drop(columns=drop_cols, errors='ignore')

    # Split features and target
    X_train_full = train_df.drop(columns=[target_col])
    y_train_full = train_df[target_col]
    X_test = test_df.drop(columns=[target_col])
    y_test = test_df[target_col]

    # Preprocessor
    preprocessor = build_transformer()
    
    # Model
    name, model = build_model(model_name)
    print(f"\nTraining model: {name} using StratifiedKFold...")

    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    acc_scores, f1_scores, roc_scores = [], [], []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_full, y_train_full), 1):
        X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
        y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

        pipeline = Pipeline([
            ("preprocess", preprocessor),
            ("feature_selector", build_feature_selector(model_name)),
            ("classifier", model)
        ])
        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_val)
        y_proba = pipeline.predict_proba(X_val)[:, 1]
        

        acc_scores.append(accuracy_score(y_val, y_pred))
        f1_scores.append(f1_score(y_val, y_pred))
        roc_scores.append(roc_auc_score(y_val, y_proba))

        print(f"Fold {fold}: Accuracy={acc_scores[-1]:.3f}, F1={f1_scores[-1]:.3f}, ROC-AUC={roc_scores[-1]:.3f}")

    results = {
        "model": name,
        "accuracy": np.mean(acc_scores),
        "f1_score": np.mean(f1_scores),
        "roc_auc": np.nanmean(roc_scores)
    }

    print(f"\nFinished training {name} across all folds.")
    print(f"Average Accuracy: {results['accuracy']:.3f}, F1: {results['f1_score']:.3f}, ROC-AUC: {results['roc_auc']:.3f}")

    return results, X_train_full, y_train_full, X_test, y_test


In [18]:
models = ["XGB", "KNN"]
for model in models:
    # train_df, test_df = data_pipeline()
    results_df, X_train, y_train, X_test, y_test = model_pipeline(model_name=model, 
        train_df=train, test_df=test, target_col="label", random_state=42
    )


Training model: XGB Classifier using StratifiedKFold...


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:16] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:16] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 1: Accuracy=0.815, F1=0.205, ROC-AUC=0.580


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:17] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:17] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 2: Accuracy=0.809, F1=0.187, ROC-AUC=0.581


...

Fold 3: Accuracy=0.815, F1=0.203, ROC-AUC=0.582


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:18] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:18] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 4: Accuracy=0.818, F1=0.209, ROC-AUC=0.577


/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:18] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/workspaces/CS3244_ML_Project/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [16:19:18] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fold 5: Accuracy=0.810, F1=0.190, ROC-AUC=0.572

Finished training XGB Classifier across all folds.
Average Accuracy: 0.813, F1: 0.199, ROC-AUC: 0.579

Training model: KNN using StratifiedKFold...
Fold 1: Accuracy=0.773, F1=0.147, ROC-AUC=0.560
Fold 2: Accuracy=0.767, F1=0.148, ROC-AUC=0.546
Fold 3: Accuracy=0.770, F1=0.167, ROC-AUC=0.562
Fold 4: Accuracy=0.769, F1=0.163, ROC-AUC=0.547
Fold 5: Accuracy=0.763, F1=0.139, ROC-AUC=0.547

Finished training KNN across all folds.
Average Accuracy: 0.769, F1: 0.153, ROC-AUC: 0.552
